In [23]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma

from langchain_community.embeddings import HuggingFaceEmbeddings


embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

docs = []
for file in ["Hybrid Work Policy 2026.pdf"]:
    loader = PyPDFLoader(f"{file}")
    docs.extend(loader.load())
db = Chroma.from_documents(
    docs,
    embedding=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"),
    persist_directory="./vectorstore"
)
db.persist()


In [24]:
def hr_policy_search(query: str):
    results = db.similarity_search(query, k=3)
    return "\n".join([r.page_content for r in results])


In [25]:
from langchain_community.tools import DuckDuckGoSearchRun

web_search = DuckDuckGoSearchRun()

In [51]:
import requests

def call_mcp_tool(query: str) -> str:
    """Call the MCP server's search_insurance_docs tool"""
    try:
        response = requests.post(
            "http://127.0.0.1:8000/search",
            json={"query": query},
            headers={"Content-Type": "application/json"},
            timeout=10
        )
        
        if response.status_code == 200:
            result = response.json()
            return result.get("result", "No results")
        else:
            return f"Error calling MCP: {response.status_code}"
    except requests.exceptions.RequestException as e:
        return f"Error connecting to MCP server: {str(e)}"

In [ ]:
import os
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),  # your Azure deployment name
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0,
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)


In [28]:
# from langchain_community.chat_models import ChatOllama

# llm = ChatOllama(
#     model="llama3",
#     temperature=0,
# )


In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.tools import Tool

# Define your tools
tools = [
    Tool(
        name="HR_Policy_Search",
        func=hr_policy_search,
        description="Search HR policy documents for information about policies, procedures, and guidelines"
    ),
    Tool(
        name="Web_Search",
        func=web_search.run,
        description="Fetch industry benchmarks, trends, and external information from the web"
    ),
    Tool(
        name="Insurance_Docs_MCP",
        func=call_mcp_tool,
        description="Query Presidio insurance Google Docs for health insurance, life insurance, and benefits information"
    )
]


# Create a ReAct prompt template
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

# Create the agent
agent = create_react_agent(llm, tools, prompt)

# Create an agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)



In [53]:
# Now invoke the agent
result = agent_executor.invoke({
    "input": "Compare our current hiring trend with industry benchmarks."
})

print(result)



> Entering new AgentExecutor chain...
Action: Web_Search
Action Input: "2024 hiring trends and benchmarks in our industry"Jan 19, 2024 · These are some of the key moments of the week from Davos 2024, from global cooperation to artificial intelligence, and climate change to global growth. Apr 14, 2025 · Renewable energy capacity grew by record amounts in 2024 – but it’s still not enough to keep us on target for our energy transition goals. Dec 17, 2024 · Health was a major focus in 2024, shaping global news and driving key discussions at the World Economic Forum. From climate change health impacts to the rise of antimicrobial … Macbook Pro M4 2024 chính hãng, trả góp 0%, trả trước 0 đồng, phụ phí 0 đồng, trợ giá lên đời đến 2 triệu. Mua ngay Mac Pro M4 (Pro, Max) nhé Jun 27, 2024 · Annual Meeting of the New Champions 2024 (AMNC24) will take place in Dalian, People’s Republic of China, on 25-27 June 2024.Thought: The initial search returned general global trends and news, but not speci

In [ ]:
result["output"]

"Industry benchmarks for 2024 indicate that over 81% of HR professionals across sectors are currently hiring, with about 11% experiencing a hiring freeze and 8% having no openings. Nearly 70% of HRs are prioritizing talent attraction this year. To directly compare our organization's hiring trend with these benchmarks, we would need specific data on our current hiring activity (such as the number of hires, hiring freezes, or open positions). If you can provide this internal data, I can offer a more precise comparison. Based on available information, the industry is generally in a strong hiring phase, especially in technology and roles related to AI and business transformation."

In [55]:
# Now invoke the agent
result = agent_executor.invoke({
    "input": "Give me details about health insurance and hospital expenses options available for employees."
})

print(result)



> Entering new AgentExecutor chain...
I should check the Presidio insurance documents for specific details about health insurance and hospital expense options available for employees.
Action: Insurance_Docs_MCP
Action Input: health insurance and hospital expenses options for employees📋 Found in insurance documents:

EMPLOYEES COMPENSATION INSURANCE POLICY

and deemed to be incorporated herein, has applied to SBI General Insurance Company

Limited (hereinafter called “the Company”) for the insu rance hereinafter contained for the

Business described in the Schedule and has paid or agreed to pay the premium stated

in the Schedule  as consideration for such insuranceAction: Insurance_Docs_MCP
Action Input: Details of health insurance coverage, hospital expenses, and benefits for employees📋 Found in insurance documents:

EMPLOYEES COMPENSATION INSURANCE POLICY

WHEREAS the Insured  by a Proposal which shall be the basis of this Contract

and deemed to be incorporated herein, has applied